In [288]:
from datetime import datetime
import sys
import os
import math
import numpy as np
import pandas as pd
import pandas_ta as ta
import datetime
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay
US_BUSINESS_DAY = CustomBusinessDay(calendar=USFederalHolidayCalendar())
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import itertools
import matplotlib.dates as mpl_dates
import yfinance as yf
#import ta as tb
from finta import TA
from os.path import exists

In [289]:
def load_historic_data(symbol, start_date_str, today_date_str, period, interval, prepost):
    try:
        df = yf.download(symbol, start=start_date_str, end=today_date_str, period=period, interval=interval, prepost=prepost)
        #  Add symbol
        df["Symbol"] = symbol
        df['high'] = df['High']
        df['low'] = df['Low']
        df['open'] = df['Open']
        df['close'] = df['Close']
        df = pd.DataFrame(df, columns = ['open','low', 'high','close'])
        return df
    except:
        print('Error loading stock data for ' + symbol)
        return None

In [290]:
def calculate_tis(df, period):
    #  Calculate ATR
    atr = TA.ATR(df, period=period)
    
    #  Calculate chandelier exits
    chandelier_info = TA.CHANDELIER(df, short_period=period, long_period=period, k=3)

    #  Add to price dataframe
    df = pd.concat([df, atr, chandelier_info], axis=1, ignore_index=False)

    return df

In [291]:
def calculate_signals(df):
    #  Long position
    df['enter_long'] = np.where((df['close'] > df['Short.']) & (df['close'].shift(1) <= df['Short.'].shift(1)), 1, 0)
    df['exit_long'] = np.where((df['close'] < df['Long.']) & (df['close'].shift(1) >= df['Long.'].shift(1)), 1, 0) 
    
    #  Short position
    df['enter_short'] = np.where((df['close'] < df['Long.']) & (df['close'].shift(1) >= df['Long.'].shift(1)), 1, 0)
    df['exit_short'] = np.where((df['close'] > df['Short.']) & (df['close'].shift(1) <= df['Short.'].shift(1)), 1, 0)
    return df

In [292]:
def execute_strategy(df, use_stop_loss):
    close_prices = df['close'].to_numpy()
    enter_long = df['enter_long'].to_numpy()
    exit_long = df['exit_long'].to_numpy()
    enter_short = df['enter_short'].to_numpy()
    exit_short = df['exit_short'].to_numpy()
    
    last_long_entry_price = 0
    last_short_entry_price = 0
    long_entry_prices = []
    long_exit_prices = []
    short_entry_prices = []
    short_exit_prices = []
    hold_long = 0
    hold_short = 0

    for i in range(len(close_prices)):
        current_price = close_prices[i]
        
        #  Enter long 
        if hold_long == 0 and enter_long[i] == 1:
            last_long_entry_price = current_price
            long_entry_prices.append(current_price)
            long_exit_prices.append(np.nan)
            hold_long = 1
        #  Exit long
        elif hold_long == 1 and exit_long[i] == 1:
            long_entry_prices.append(np.nan)
            long_exit_prices.append(current_price)
            hold_long = 0
        #  Long stop loss
        elif use_stop_loss and hold_long == 1 and current_price < last_long_entry_price:
            long_entry_prices.append(np.nan)
            long_exit_prices.append(current_price)
            hold_long = 0
        else:
            #  Neither entry nor exit
            long_entry_prices.append(np.nan)
            long_exit_prices.append(np.nan)

        #  Enter Short 
        if hold_short == 0 and enter_short[i] == 1:
            last_short_entry_price = current_price
            short_entry_prices.append(current_price)
            short_exit_prices.append(np.nan)
            hold_short = 1
        #  Short stop loss
        elif use_stop_loss and hold_short == 1 and current_price > last_short_entry_price:
            short_entry_prices.append(np.nan)
            short_exit_prices.append(current_price)
            hold_short = 0
        #  Exit short
        elif hold_short == 1 and exit_short[i] == 1:
            short_entry_prices.append(np.nan)
            short_exit_prices.append(current_price)
            hold_short = 0
        else:
            #  Neither entry nor exit
            short_entry_prices.append(np.nan)
            short_exit_prices.append(np.nan)

    return long_entry_prices, long_exit_prices, short_entry_prices, short_exit_prices

In [293]:
def plot_graph(symbol, df, long_entry_prices, long_exit_prices, short_entry_prices, short_exit_prices):
    fig = make_subplots(rows=2, cols=1, subplot_titles=['close','ATR'])
    #df = df.reset_index()
    print(long_entry_prices)
    print(df.head())
    #  Plot close price
    fig.add_trace(go.Line(x = df.index, y = df['close'], line=dict(color="blue", width=1), name="Close"), row = 1, col = 1) 
    
    #  Plot ATR
    fig.add_trace(go.Line(x = df.index, y = df['30 period ATR'], line=dict(color="blue", width=1), name="ATR"), row = 2, col = 1)

    #  Long markers
    fig.add_trace(go.Scatter(x=df.index, y=long_entry_prices, marker_symbol="arrow-up", marker=dict(
        color='green',size=9
    ),mode='markers',name='Enter Long'))
    fig.add_trace(go.Scatter(x=df.index, y=long_exit_prices, marker_symbol="arrow-down", marker=dict(
        color='red', size=9
    ),mode='markers',name='Exit Long'))

    #  Short markers
    fig.add_trace(go.Scatter(x=df.index + 8, y=short_entry_prices, marker_symbol="arrow-down", marker=dict(
        color='#8eb028',size=9
    ),mode='markers',name='Enter Short'))
    fig.add_trace(go.Scatter(x=df.index + 8, y=short_exit_prices, marker_symbol="arrow-up", marker=dict(
        color='#7b32a8', size=9
    ),mode='markers',name='Exit Short'))

    
    fig.update_layout(
        title={'text':f"{symbol} with Chandelier Exits", 'x':0.5},
        autosize=False,
        width=800,height=600)
    fig.update_yaxes(range=[0,1000000000],secondary_y=True)
    fig.update_yaxes(visible=False, secondary_y=True)  #hide range slider
    
    fig.show()
    fig.write_image('plot.png', format="png")

In [294]:
#  Assuming reinvesting the proceeds
def calculate_profit(start_investment, long_entry_prices, long_exit_prices, short_entry_prices, short_exit_prices):
    hold_long, hold_short = 0, 0
    available_funds = start_investment
    cost_long, cost_short = 0, 0
    num_stocks_long, num_stocks_short = 0, 0
    proceeds_short = 0
    profit = 0
    for i in range(len(long_entry_prices)):
        #  Go long
        current_entry_price_long = long_entry_prices[i]
        current_exit_price_long = long_exit_prices[i]
        if not math.isnan(current_entry_price_long) and hold_long == 0:
            num_stocks_long = available_funds / current_entry_price_long
            cost_long = num_stocks_long * current_entry_price_long
            hold_long = 1

        elif hold_long == 1 and not math.isnan(current_exit_price_long):
            hold_long = 0
            proceeds = num_stocks_long * current_exit_price_long
            profit += proceeds - cost_long

        #  Go short
        current_entry_price_short = short_entry_prices[i]
        current_exit_price_short = short_exit_prices[i]
        if not math.isnan(current_entry_price_short) and hold_short == 0:
            num_stocks_short = available_funds / current_entry_price_short
            proceeds_short = num_stocks_short * current_entry_price_short
            hold_short = 1
        elif hold_short == 1 and not math.isnan(current_exit_price_short):
            hold_short = 0
            cost_short = num_stocks_short * current_exit_price_short
            profit += proceeds_short - cost_short

    return math.trunc(profit)

In [295]:
def build_data_file_path(symbol, interval):
    file_name = f"{today_date_str}_{symbol}_{interval}.csv"
    file_path = os.path.join("C:\\dev\\trading\\tradesystem1\\data\\crypto",file_name)
    return file_path

In [296]:
def perform_evaluation(symbol, config):
    print(f"Eval for {symbol}, data interval: {config['data_interval']}, Use Stop Loss: {config['use_stop_loss']}, ATR period: {config['atr_period']}")
    
    file_path = build_data_file_path(symbol, config['data_interval'])
    if not exists(file_path):
        print(f"Error loading file {file_path}")
        return
    
    #  load df
    df = pd.read_csv(file_path)
    df = df.tail(600)
    
    #  Calculate chandelier exits and signals
    df = calculate_tis(df, config['atr_period'])
    df = calculate_signals(df)
    
    #  Run strategy
    long_entry_prices, long_exit_prices, short_entry_prices, short_exit_prices = execute_strategy(df, config['use_stop_loss'])

    #  Calculate profit
    start_investment = 100
    profit = calculate_profit(start_investment, long_entry_prices, long_exit_prices, short_entry_prices, short_exit_prices)
    
    return profit

In [297]:
def fetch_data_for_intervals(symbols, data_interval_array):
    prepost = True
    #  NOTE: 7 days is the max allowed
    days = datetime.timedelta(7)
    start_date = today - days
    start_date_str = datetime.datetime.strftime(start_date, "%Y-%m-%d")#  Coins to download
    
    for symbol in symbols:
        for data_interval in data_interval_array:
            file_path = build_data_file_path(symbol, data_interval)
            if not exists(file_path):
                #  Load data and store
                df = load_historic_data(symbol, start_date_str, today_date_str, '1d', data_interval, prepost)
                df.to_csv(file_path)

In [298]:
# MAIN
today = datetime.date.today()
today_date_str = today.strftime("%Y-%m-%d")
    
symbols = ['BTC-USD','ETH-USD','LTC-USD','DOGE-USD']

#  Fetch data from Yahoo finance and store for our tests
# Valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
# Fetch data by interval (including intraday if period < 60 days)
        # valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
data_interval_array = ['1m','2m','5m','15m','30m','60m','90m','1h']
fetch_data_for_intervals(symbols, data_interval_array)

use_stop_loss_array = [True, False]
atr_period_min = 10
atr_period_max = 30

#  Test all combinations or input parameters
config = {}
profit_df = pd.DataFrame()
for data_interval in data_interval_array:
    config['data_interval'] = data_interval
    for use_stop_loss in use_stop_loss_array:
        config['use_stop_loss'] = use_stop_loss
        for atr_period in range(atr_period_min, atr_period_max):
            config['atr_period'] = atr_period
            for symbol in symbols:
                profit = perform_evaluation(symbol, config)
                config_combination = f"data_interval-{data_interval}-atr_period-{atr_period}-use_stop_loss-{use_stop_loss}"
                new_profit_row = pd.DataFrame({'symbol': [symbol], 'combinations': [config_combination], 'profit':[profit]})
                profit_df = pd.concat([profit_df, new_profit_row], axis=0, ignore_index=True)

#  Aggregate results by config combinations and sort to get best combinations
results_df = profit_df.groupby('combinations')['profit'].agg(['mean'])
print(results_df.info)
results_df = results_df.sort_values(by='mean',ascending=False)
results_df.to_csv('results.csv')

print('Top 10 Results:')
print(results_df.head(10))


Eval for BTC-USD, data interval: 1m, Use Stop Loss: True, ATR period: 10
Eval for ETH-USD, data interval: 1m, Use Stop Loss: True, ATR period: 10
Eval for LTC-USD, data interval: 1m, Use Stop Loss: True, ATR period: 10
Eval for DOGE-USD, data interval: 1m, Use Stop Loss: True, ATR period: 10
Eval for BTC-USD, data interval: 1m, Use Stop Loss: True, ATR period: 11
Eval for ETH-USD, data interval: 1m, Use Stop Loss: True, ATR period: 11
Eval for LTC-USD, data interval: 1m, Use Stop Loss: True, ATR period: 11
Eval for DOGE-USD, data interval: 1m, Use Stop Loss: True, ATR period: 11
Eval for BTC-USD, data interval: 1m, Use Stop Loss: True, ATR period: 12
Eval for ETH-USD, data interval: 1m, Use Stop Loss: True, ATR period: 12
Eval for LTC-USD, data interval: 1m, Use Stop Loss: True, ATR period: 12
Eval for DOGE-USD, data interval: 1m, Use Stop Loss: True, ATR period: 12
Eval for BTC-USD, data interval: 1m, Use Stop Loss: True, ATR period: 13
Eval for ETH-USD, data interval: 1m, Use Stop Lo

Eval for LTC-USD, data interval: 1m, Use Stop Loss: False, ATR period: 20
Eval for DOGE-USD, data interval: 1m, Use Stop Loss: False, ATR period: 20
Eval for BTC-USD, data interval: 1m, Use Stop Loss: False, ATR period: 21
Eval for ETH-USD, data interval: 1m, Use Stop Loss: False, ATR period: 21
Eval for LTC-USD, data interval: 1m, Use Stop Loss: False, ATR period: 21
Eval for DOGE-USD, data interval: 1m, Use Stop Loss: False, ATR period: 21
Eval for BTC-USD, data interval: 1m, Use Stop Loss: False, ATR period: 22
Eval for ETH-USD, data interval: 1m, Use Stop Loss: False, ATR period: 22
Eval for LTC-USD, data interval: 1m, Use Stop Loss: False, ATR period: 22
Eval for DOGE-USD, data interval: 1m, Use Stop Loss: False, ATR period: 22
Eval for BTC-USD, data interval: 1m, Use Stop Loss: False, ATR period: 23
Eval for ETH-USD, data interval: 1m, Use Stop Loss: False, ATR period: 23
Eval for LTC-USD, data interval: 1m, Use Stop Loss: False, ATR period: 23
Eval for DOGE-USD, data interval: 1

Eval for DOGE-USD, data interval: 2m, Use Stop Loss: False, ATR period: 11
Eval for BTC-USD, data interval: 2m, Use Stop Loss: False, ATR period: 12
Eval for ETH-USD, data interval: 2m, Use Stop Loss: False, ATR period: 12
Eval for LTC-USD, data interval: 2m, Use Stop Loss: False, ATR period: 12
Eval for DOGE-USD, data interval: 2m, Use Stop Loss: False, ATR period: 12
Eval for BTC-USD, data interval: 2m, Use Stop Loss: False, ATR period: 13
Eval for ETH-USD, data interval: 2m, Use Stop Loss: False, ATR period: 13
Eval for LTC-USD, data interval: 2m, Use Stop Loss: False, ATR period: 13
Eval for DOGE-USD, data interval: 2m, Use Stop Loss: False, ATR period: 13
Eval for BTC-USD, data interval: 2m, Use Stop Loss: False, ATR period: 14
Eval for ETH-USD, data interval: 2m, Use Stop Loss: False, ATR period: 14
Eval for LTC-USD, data interval: 2m, Use Stop Loss: False, ATR period: 14
Eval for DOGE-USD, data interval: 2m, Use Stop Loss: False, ATR period: 14
Eval for BTC-USD, data interval: 2

Eval for LTC-USD, data interval: 5m, Use Stop Loss: True, ATR period: 19
Eval for DOGE-USD, data interval: 5m, Use Stop Loss: True, ATR period: 19
Eval for BTC-USD, data interval: 5m, Use Stop Loss: True, ATR period: 20
Eval for ETH-USD, data interval: 5m, Use Stop Loss: True, ATR period: 20
Eval for LTC-USD, data interval: 5m, Use Stop Loss: True, ATR period: 20
Eval for DOGE-USD, data interval: 5m, Use Stop Loss: True, ATR period: 20
Eval for BTC-USD, data interval: 5m, Use Stop Loss: True, ATR period: 21
Eval for ETH-USD, data interval: 5m, Use Stop Loss: True, ATR period: 21
Eval for LTC-USD, data interval: 5m, Use Stop Loss: True, ATR period: 21
Eval for DOGE-USD, data interval: 5m, Use Stop Loss: True, ATR period: 21
Eval for BTC-USD, data interval: 5m, Use Stop Loss: True, ATR period: 22
Eval for ETH-USD, data interval: 5m, Use Stop Loss: True, ATR period: 22
Eval for LTC-USD, data interval: 5m, Use Stop Loss: True, ATR period: 22
Eval for DOGE-USD, data interval: 5m, Use Stop L

Eval for DOGE-USD, data interval: 15m, Use Stop Loss: True, ATR period: 10
Eval for BTC-USD, data interval: 15m, Use Stop Loss: True, ATR period: 11
Eval for ETH-USD, data interval: 15m, Use Stop Loss: True, ATR period: 11
Eval for LTC-USD, data interval: 15m, Use Stop Loss: True, ATR period: 11
Eval for DOGE-USD, data interval: 15m, Use Stop Loss: True, ATR period: 11
Eval for BTC-USD, data interval: 15m, Use Stop Loss: True, ATR period: 12
Eval for ETH-USD, data interval: 15m, Use Stop Loss: True, ATR period: 12
Eval for LTC-USD, data interval: 15m, Use Stop Loss: True, ATR period: 12
Eval for DOGE-USD, data interval: 15m, Use Stop Loss: True, ATR period: 12
Eval for BTC-USD, data interval: 15m, Use Stop Loss: True, ATR period: 13
Eval for ETH-USD, data interval: 15m, Use Stop Loss: True, ATR period: 13
Eval for LTC-USD, data interval: 15m, Use Stop Loss: True, ATR period: 13
Eval for DOGE-USD, data interval: 15m, Use Stop Loss: True, ATR period: 13
Eval for BTC-USD, data interval: 1

...

Eval for ETH-USD, data interval: 30m, Use Stop Loss: False, ATR period: 13
Eval for LTC-USD, data interval: 30m, Use Stop Loss: False, ATR period: 13
Eval for DOGE-USD, data interval: 30m, Use Stop Loss: False, ATR period: 13
Eval for BTC-USD, data interval: 30m, Use Stop Loss: False, ATR period: 14
Eval for ETH-USD, data interval: 30m, Use Stop Loss: False, ATR period: 14
Eval for LTC-USD, data interval: 30m, Use Stop Loss: False, ATR period: 14
Eval for DOGE-USD, data interval: 30m, Use Stop Loss: False, ATR period: 14
Eval for BTC-USD, data interval: 30m, Use Stop Loss: False, ATR period: 15
Eval for ETH-USD, data interval: 30m, Use Stop Loss: False, ATR period: 15
Eval for LTC-USD, data interval: 30m, Use Stop Loss: False, ATR period: 15
Eval for DOGE-USD, data interval: 30m, Use Stop Loss: False, ATR period: 15
Eval for BTC-USD, data interval: 30m, Use Stop Loss: False, ATR period: 16
Eval for ETH-USD, data interval: 30m, Use Stop Loss: False, ATR period: 16
Eval for LTC-USD, data

Eval for ETH-USD, data interval: 60m, Use Stop Loss: True, ATR period: 26
Eval for LTC-USD, data interval: 60m, Use Stop Loss: True, ATR period: 26
Eval for DOGE-USD, data interval: 60m, Use Stop Loss: True, ATR period: 26
Eval for BTC-USD, data interval: 60m, Use Stop Loss: True, ATR period: 27
Eval for ETH-USD, data interval: 60m, Use Stop Loss: True, ATR period: 27
Eval for LTC-USD, data interval: 60m, Use Stop Loss: True, ATR period: 27
Eval for DOGE-USD, data interval: 60m, Use Stop Loss: True, ATR period: 27
Eval for BTC-USD, data interval: 60m, Use Stop Loss: True, ATR period: 28
Eval for ETH-USD, data interval: 60m, Use Stop Loss: True, ATR period: 28
Eval for LTC-USD, data interval: 60m, Use Stop Loss: True, ATR period: 28
Eval for DOGE-USD, data interval: 60m, Use Stop Loss: True, ATR period: 28
Eval for BTC-USD, data interval: 60m, Use Stop Loss: True, ATR period: 29
Eval for ETH-USD, data interval: 60m, Use Stop Loss: True, ATR period: 29
Eval for LTC-USD, data interval: 60

Eval for ETH-USD, data interval: 90m, Use Stop Loss: True, ATR period: 15
Eval for LTC-USD, data interval: 90m, Use Stop Loss: True, ATR period: 15
Eval for DOGE-USD, data interval: 90m, Use Stop Loss: True, ATR period: 15
Eval for BTC-USD, data interval: 90m, Use Stop Loss: True, ATR period: 16
Eval for ETH-USD, data interval: 90m, Use Stop Loss: True, ATR period: 16
Eval for LTC-USD, data interval: 90m, Use Stop Loss: True, ATR period: 16
Eval for DOGE-USD, data interval: 90m, Use Stop Loss: True, ATR period: 16
Eval for BTC-USD, data interval: 90m, Use Stop Loss: True, ATR period: 17
Eval for ETH-USD, data interval: 90m, Use Stop Loss: True, ATR period: 17
Eval for LTC-USD, data interval: 90m, Use Stop Loss: True, ATR period: 17
Eval for DOGE-USD, data interval: 90m, Use Stop Loss: True, ATR period: 17
Eval for BTC-USD, data interval: 90m, Use Stop Loss: True, ATR period: 18
Eval for ETH-USD, data interval: 90m, Use Stop Loss: True, ATR period: 18
Eval for LTC-USD, data interval: 90

Eval for ETH-USD, data interval: 90m, Use Stop Loss: False, ATR period: 24
Eval for LTC-USD, data interval: 90m, Use Stop Loss: False, ATR period: 24
Eval for DOGE-USD, data interval: 90m, Use Stop Loss: False, ATR period: 24
Eval for BTC-USD, data interval: 90m, Use Stop Loss: False, ATR period: 25
Eval for ETH-USD, data interval: 90m, Use Stop Loss: False, ATR period: 25
Eval for LTC-USD, data interval: 90m, Use Stop Loss: False, ATR period: 25
Eval for DOGE-USD, data interval: 90m, Use Stop Loss: False, ATR period: 25
Eval for BTC-USD, data interval: 90m, Use Stop Loss: False, ATR period: 26
Eval for ETH-USD, data interval: 90m, Use Stop Loss: False, ATR period: 26
Eval for LTC-USD, data interval: 90m, Use Stop Loss: False, ATR period: 26
Eval for DOGE-USD, data interval: 90m, Use Stop Loss: False, ATR period: 26
Eval for BTC-USD, data interval: 90m, Use Stop Loss: False, ATR period: 27
Eval for ETH-USD, data interval: 90m, Use Stop Loss: False, ATR period: 27
Eval for LTC-USD, data

Eval for LTC-USD, data interval: 1h, Use Stop Loss: False, ATR period: 12
Eval for DOGE-USD, data interval: 1h, Use Stop Loss: False, ATR period: 12
Eval for BTC-USD, data interval: 1h, Use Stop Loss: False, ATR period: 13
Eval for ETH-USD, data interval: 1h, Use Stop Loss: False, ATR period: 13
Eval for LTC-USD, data interval: 1h, Use Stop Loss: False, ATR period: 13
Eval for DOGE-USD, data interval: 1h, Use Stop Loss: False, ATR period: 13
Eval for BTC-USD, data interval: 1h, Use Stop Loss: False, ATR period: 14
Eval for ETH-USD, data interval: 1h, Use Stop Loss: False, ATR period: 14
Eval for LTC-USD, data interval: 1h, Use Stop Loss: False, ATR period: 14
Eval for DOGE-USD, data interval: 1h, Use Stop Loss: False, ATR period: 14
Eval for BTC-USD, data interval: 1h, Use Stop Loss: False, ATR period: 15
Eval for ETH-USD, data interval: 1h, Use Stop Loss: False, ATR period: 15
Eval for LTC-USD, data interval: 1h, Use Stop Loss: False, ATR period: 15
Eval for DOGE-USD, data interval: 1